In [ ]:
# import torch
# import torch.nn as nn
# kernel_List = [12, 4, 4, 4, 4, 4]
# channel_List = [128, 256, 512, 512, 512, 400]

# class ConvBlock(nn.Module):
#     def __init__(self, in_channel, out_channel, kernel_sz, padding, stride = 2) -> None:
#         super().__init__()
#         self.conv = nn.Conv2d(in_channel, out_channel, kernel_sz, stride, padding)
#         self.bn = nn.BatchNorm2d(out_channel)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout()
#     def forward(self, x):
#         x = self.relu(self.bn(self.conv(x)))
#         x = self.dropout(x)
#         return x

# def get_convBlocks(in_channel):
#     layerNum = len(kernel_List)
#     blocks = []
#     blocks.append(ConvBlock(in_channel, channel_List[0], kernel_List[0], int(kernel_List[0] / 2 - 1)))
#     for i in range(1, layerNum):
#         blocks.append(ConvBlock(channel_List[i-1], channel_List[i], kernel_List[i], int(kernel_List[i] / 2 - 1)))
#     return blocks

# class DeepFold(nn.Module):
#     def __init__(self, in_channel) -> None:
#         super().__init__()
#         self.convLayer = nn.Sequential(*get_convBlocks(in_channel))
    
#     # [batch_size, 3, 256, 256]
#     def forward(self, x):
#         x = self.convLayer(x) # [batch_size, 400, 4, 4]
#         x = torch.diagonal(x, dim1=2, dim2=3) # [batch_size, 400, 4]
#         x = torch.mean(x, dim= 2) # [batch_size, 400]

#         normValue = torch.norm(x, dim = 1) # norm_value [batch_size]
#         # print(normValue.shape)
#         # [400, batch_size]  除法要求最后一维要和norm_value维度匹配
#         x = x.permute(1, 0)
#         # [400, batch_size] 已经正则化
#         x = torch.div(x, normValue)

#         # [batch_size, 400]
#         x = x.permute(1, 0)
#         return x


In [2]:
import random
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn
import os

In [3]:
kernel_List = [12, 4, 4, 4, 4, 4]
channel_List = [128, 256, 512, 512, 512, 400]

class ConvBlock(nn.Module):   
    def __init__(self, in_channel, out_channel, kernel_sz, padding, stride = 2) -> None:
        super().__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_sz, stride, padding)
        self.bn = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()

    def forward(self, x):
        x = self.relu(self.bn(self.conv(x)))
        x = self.dropout(x)
        return x

def get_convBlocks(in_channel):
    layerNum = len(kernel_List)
    blocks = []
    blocks.append(ConvBlock(in_channel, channel_List[0], kernel_List[0], int(kernel_List[0] / 2 - 1)))
    for i in range(1, layerNum):
        blocks.append(ConvBlock(channel_List[i-1], channel_List[i], kernel_List[i], int(kernel_List[i] / 2 - 1)))
    return blocks

class DeepFold(nn.Module):
    def __init__(self, in_channel) -> None:
        super().__init__()
        self.convLayer = nn.Sequential(*get_convBlocks(in_channel))
    
    # [batch_size, 3, 256, 256] -> [batch_size, 400]
    def forward(self, x):
        x = self.convLayer(x) # [batch_size, 400, 4, 4]
        x = torch.diagonal(x, dim1=2, dim2=3) # [batch_size, 400, 4]
        x = torch.mean(x, dim= 2) # [batch_size, 400]
        x = F.normalize(x)
        return x

In [4]:
# 原数据应是取逆扩充channel后的距离矩阵，而且inf项被替换
def build_transform(in_channel):
    train_tfm = T.Compose(
        [
            T.Resize((256, 256)),
            # 取逆矩阵 扩充channel
            # Pretfm(in_channel),
            #是否需要数据增强 保留一个问号
            # 层归一化
            # nn.LayerNorm((in_channel, 256, 256)) # 不能经过LayNorm等网络层，不然输出数据 requires_grad = True,从而报错，原始数据应该为False
            T.Normalize(mean=[0.0068, 0.0003, 2.3069e-05], std=[0.0140, 0.0015, 0.0002])
        ]
    )
    return train_tfm

In [5]:
class Train_set(torch.utils.data.Dataset):

    def __init__(self, dict_data, id_list, tfm) -> None:
        super().__init__()
        self.tensor_list = []
        for id, label in id_list:
            # 在蛋白质数据库文件查找 id.npy
            # feature = torch.from_numpy(np.load(dir+id+".npy", allow_pickle=True))
            feature = torch.from_numpy(dict_data[id])
            feature = feature.to(torch.float) # torch.float = torch.float32
            label = float(label)
            self.tensor_list.append((feature,
                                        label)
                                        )
        self.tfm = tfm

    def __getitem__(self, idx :int):
        y = self.tensor_list[idx][0]
        y = self.tfm(y)
        label = torch.tensor(self.tensor_list[idx][1], dtype=torch.float32)
        return y, label

    def __len__(self):
        return len(self.tensor_list)


In [6]:
# 获取左侧一列id 对应的右侧id_list
def get_id_list(pair_path):
    id_list = []
    with open(pair_path, "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            line1= lines.split('\t')[0]
            line2 = lines.split('\t')[1].split("\n")[0]
            id_list.append((line1, line2))
    return id_list

In [7]:
# # 获取id对应的distance_matrix
def get_feature(dict_data, id, tfm):
    feature = torch.from_numpy(dict_data[id])
    feature = feature.to(torch.float)
    feature = tfm(feature)
    feature = feature.unsqueeze(0)
    return feature

In [8]:
class Max_margin_loss(nn.Module):
    def __init__(self, K, m) -> None:
        super().__init__()
        self.K = K
        self.m = m
   
    def forward(self, fingerpvec1, fingerpvec2, label):
        posi_vec_list = []
        nega_vec_list = []
        for number_inbatch in range(fingerpvec2.shape[0]):
            if label[number_inbatch] == 0:
                nega_vec_list.append(fingerpvec2[number_inbatch])
            elif label[number_inbatch] == 1:
                posi_vec_list.append(fingerpvec2[number_inbatch])
            else:
                print("ERROR")

        posi_cos_smi_list = []
        nega_cos_smi_list = []
        for posi_vec in posi_vec_list:
            # print("posi_vec: ", posi_vec.shape)
            posi_cos_smi_list.append(F.cosine_similarity(fingerpvec1, posi_vec, dim = 0))
        for nega_vec in nega_vec_list:
            nega_cos_smi_list.append(F.cosine_similarity(fingerpvec1, nega_vec, dim = 0))

        posi_cos_smi_list.sort() # 升序排序 选最小
        nega_cos_smi_list.sort(reverse=True) # 降序排序 选最大
        posi_cos = posi_cos_smi_list[0] # 只选取一个正例
        loss = 0
        for i in range(self.K):
            nega_cos = nega_cos_smi_list[i]
            loss += max(0, nega_cos - posi_cos + self.m)
        return loss

In [9]:
def valid_data(best_acc, valid_acc_ls, dict_data, validIDlist, DFold_model, train_tfm, batch_size, device, epoch, K=5):
    
    valid_pair_dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"
    DFold_model.eval()
    acc_num = 0
    for id in validIDlist:
        feature1 = get_feature(dict_data, id, train_tfm)
        feature1 = feature1.to(device)

        id_list = get_id_list(valid_pair_dir + id +".txt")
        train_ds = Train_set(dict_data, id_list, train_tfm)
        train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)

        topKList = []
        for feature2, label in train_dl:
            with torch.no_grad():
                fingerpvec1 = DFold_model(feature1)
            feature2 = feature2.to(device)
            label = label.to(device)
            fingerBatch = fingerpvec1
            for b_i in range(feature2.shape[0] - 1):
                fingerBatch = torch.cat((fingerBatch,fingerpvec1), dim = 0)
            with torch.no_grad():
                fingerpvec2 = DFold_model(feature2)

            cos_smi_batch = F.cosine_similarity(fingerBatch, fingerpvec2, dim=-1)
            for cos_smi_idx in range(cos_smi_batch.shape[0]):
                if(len(topKList) < K):
                    topKList.append((cos_smi_batch[cos_smi_idx], label[cos_smi_idx]))
                else:
                    min_value = min(topKList)
                    if (cos_smi_batch[cos_smi_idx], label[cos_smi_idx]) > min_value:
                        min_idx = topKList.index(min_value)
                        topKList[min_idx] = (cos_smi_batch[cos_smi_idx], label[cos_smi_idx])
        acc_flag = False
        for _, label in topKList:
            if label == 1:
                acc_flag = True
        if acc_flag:
            acc_num += 1

    acc = acc_num / len(validIDlist)
    valid_acc_ls.append(acc)
    print(f"Epoch: {epoch} | acc_num: {acc_num} | total_num: {len(validIDlist)} | acc: {acc_num / len(validIDlist):.4f}")
    if acc > best_acc:
        best_acc = acc
        torch.save(DFold_model.state_dict(), "/home/wngys/lab/DeepFold/model/best_model.pt")
        print(f"saving best model with acc: {best_acc:.4f}")    
    DFold_model.train()
    return best_acc

In [ ]:
# 训练过程

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DFold_model = DeepFold(in_channel = 3)
DFold_model.to(device)

train_tfm = build_transform(in_channel = 3)
optimizer = torch.optim.SGD(DFold_model.parameters(), lr = 1e-4)
lossF = Max_margin_loss(K = 10, m = 0.1)

total_epochs = 10
batch_size = 64

train_loss_ls = []
valid_acc_ls = []

pair_dir = "/home/wngys/lab/DeepFold/pair/train_pair_bool_90/"  
valid_pair_dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"

trainIDlist = np.load("/home/wngys/lab/DeepFold/pair/train.npy", allow_pickle=True)
random.shuffle(trainIDlist)
validIDlist = np.load("/home/wngys/lab/DeepFold/pair/valid.npy", allow_pickle=True)
random.shuffle(validIDlist)
validIDlist = validIDlist[:100]

dict_data = np.load("/home/wngys/lab/DeepFold/distance_matrix_r/matrix_data.npy", allow_pickle=True).tolist()
resume_dir = None
# resume_dir = "/home/wngys/lab/DeepFold/model/model_5.pt"
if resume_dir is not None:
    chkp = torch.load("/home/wngys/lab/DeepFold/model/model_5.pt")
    st_epoch = chkp["epoch"]
    best_acc = chkp["best_acc"]
    train_loss_ls.extend(chkp["train_loss_ls"])
    valid_acc_ls.extend(chkp["valid_acc_ls"])
    DFold_model.load_state_dict(chkp["model_param"])
    optimizer.load_state_dict(chkp["optim_param"])
    validIDlist = chkp["valid_id_list"]
else:
    st_epoch = 0
    best_acc = 0


for epoch in range(st_epoch, total_epochs):
    # 遍历左侧一列集合每一个Protein ID
    DFold_model.train()
    total_train_loss = 0

    # random.shuffle(trainIDlist)
    for id_idx, id in enumerate(trainIDlist[:400]):
        feature1 = get_feature(dict_data, id, train_tfm)
        feature1 = feature1.to(device)

        id_list = get_id_list(pair_dir + id +".txt")
        train_ds = Train_set(dict_data, id_list, train_tfm)
        train_dl = DataLoader(train_ds, batch_size, shuffle=False, num_workers=2, pin_memory=True)

        IDtotalLoss = 0
        for feature2, label in train_dl:
            fingerpvec1 = DFold_model(feature1)
            fingerpvec1 = fingerpvec1.squeeze(0)
            feature2 = feature2.to(device)
            label = label.to(device)
            fingerpvec2 = DFold_model(feature2)
            
            loss = lossF(fingerpvec1, fingerpvec2, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            IDtotalLoss += loss.item()
        
        total_train_loss += IDtotalLoss

        print(f"Epoch: {epoch} | IDidx: {id_idx} | queryID: {id} | avg_loss: {IDtotalLoss / len(train_ds):.4f} | pair_num: {len(train_ds)}")

        if (id_idx + 1) % 200 == 0:
            best_acc = valid_data(best_acc, valid_acc_ls, dict_data, validIDlist, DFold_model, train_tfm, batch_size, device, epoch, K=5)

    train_loss_ls.append(total_train_loss)
    print(f"Epoch: {epoch} | total_loss: {total_train_loss:.4f}")

    chkp = {
        "epoch": epoch+1,
        "best_acc": best_acc,
        "model_param": DFold_model.state_dict(),
        "optim_param": optimizer.state_dict(),
        "train_loss_ls": train_loss_ls,
        "valid_acc_ls": valid_acc_ls,
        "valid_id_list": validIDlist
    }
    torch.save(chkp, "/home/wngys/lab/DeepFold/model/" + f"model_{epoch}.pt")


In [10]:
# 测试过程
# 测试集前五百ID 157/500 训练后的模型 acc 31.4% | 随机模型 226/500 acc 45%
# 训练集前五百ID 160/500 acc 32%  | 随机模型 242/500 acc 48%
# 完全没有学到东西
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4, 1, 2, 3"
device_ids = [0, 1, 2, 3]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DFold_model = DeepFold(in_channel = 3)
# DFold_model.to(device)
DFold_model = nn.DataParallel(DFold_model, device_ids).to(device)
# chkp = torch.load("/home/wngys/lab/DeepFold/model/model_10_400_100/model_9.pt")
# DFold_model.load_state_dict(chkp["model_param"])

train_tfm = build_transform(in_channel = 3)

test_pair_dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"
testIDlist = np.load("/home/wngys/lab/DeepFold/pair/train.npy", allow_pickle=True)[:500]
dict_data = np.load("/home/wngys/lab/DeepFold/distance_matrix_r/matrix_data.npy", allow_pickle=True).tolist()

K = 5
batch_size = 64
DFold_model.eval()

acc_num = 0
acc_ID = []
for i, id in enumerate(testIDlist):
    if (i+1) % 100 == 0:
        print(i+1)
    feature1 = get_feature(dict_data, id, train_tfm)
    feature1 = feature1.to(device)

    id_list = get_id_list(test_pair_dir + id +".txt")
    train_ds = Train_set(dict_data, id_list, train_tfm)
    train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)

    topKList = []
    for feature2, label in train_dl:
        with torch.no_grad():
            fingerpvec1 = DFold_model(feature1)
        feature2 = feature2.to(device)
        label = label.to(device)
        fingerBatch = fingerpvec1
        for b_i in range(feature2.shape[0] - 1):
            fingerBatch = torch.cat((fingerBatch,fingerpvec1), dim = 0)
        with torch.no_grad():
            fingerpvec2 = DFold_model(feature2)

        cos_smi_batch = F.cosine_similarity(fingerBatch, fingerpvec2, dim=-1)
        for cos_smi_idx in range(cos_smi_batch.shape[0]):
            if(len(topKList) < K):
                topKList.append((cos_smi_batch[cos_smi_idx], label[cos_smi_idx]))
            else:
                min_value = min(topKList)
                if (cos_smi_batch[cos_smi_idx], label[cos_smi_idx]) > min_value:
                    min_idx = topKList.index(min_value)
                    topKList[min_idx] = (cos_smi_batch[cos_smi_idx], label[cos_smi_idx])
    acc_flag = False
    for _, label in topKList:
        if label == 1:
            acc_flag = True
    if acc_flag:
        acc_num += 1
        acc_ID.append(id)

acc = acc_num / len(testIDlist)

print(f"acc_num: {acc_num} | total: {len(testIDlist)} | acc: {acc:.4f}")


100
200
300
400
500
acc_num: 242 | total: 500 | acc: 0.4840


In [13]:
Dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"

print(len(acc_ID))   
# for id in acc_ID:
#     posi_data = []
#     nega_data = []

#     with open(Dir+id+".txt", "r") as f_r:
#         while True:
#             lines = f_r.readline()
#             if not lines:
#                 break
#             # id2 = lines.split('\t')[0]
#             flag = lines.split('\t')[1].split("\n")[0]
#             # print(type(flag))
#             if flag == "1":
#                 posi_data.append(lines)
#             else:
#                 nega_data.append(lines)
    
#     posi_num = len(posi_data)
#     nega_num = len(nega_data)
#     total_num = posi_num + nega_num

#     print(f"{id} | {posi_num} | {nega_num} | {total_num}")

229


In [14]:

Dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"

    
for id in acc_ID:
    posi_data = []
    nega_data = []

    with open(Dir+id+".txt", "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            # id2 = lines.split('\t')[0]
            flag = lines.split('\t')[1].split("\n")[0]
            # print(type(flag))
            if flag == "1":
                posi_data.append(lines)
            else:
                nega_data.append(lines)
    
    posi_num = len(posi_data)
    nega_num = len(nega_data)
    total_num = posi_num + nega_num

    print(f"{id} | {posi_num} | {nega_num} | {total_num}")



d1zcha1 | 9 | 887 | 896
d1mj4a_ | 3 | 614 | 617
d1u9da1 | 11 | 1794 | 1805
d1rtqa_ | 12 | 2461 | 2473
d1pp9a1 | 4 | 1584 | 1588
d1su3a2 | 1 | 275 | 276
d1uewa1 | 17 | 253 | 270
d2cura2 | 8 | 34 | 42
d2gqfa2 | 1 | 269 | 270
d1oz2a1 | 5 | 279 | 284
d4a63b2 | 5 | 131 | 136
d2glia1 | 3 | 4412 | 4415
d1y9ka1 | 31 | 1670 | 1701
d3zhga_ | 2 | 84 | 86
d2zxea4 | 2 | 660 | 662
d1wspa1 | 2 | 825 | 827
d2r55a1 | 1 | 658 | 659
d2uyya1 | 4 | 1582 | 1586
d4agka_ | 9 | 25 | 34
d1aqza_ | 2 | 225 | 227
d2x1la2 | 2 | 505 | 507
d4yw2a1 | 6 | 63 | 69
d2cpwa1 | 10 | 1032 | 1042
d1v5ua1 | 17 | 503 | 520
d3o63a1 | 41 | 456 | 497
d2rg7a_ | 4 | 601 | 605
d1ow0a1 | 10 | 1664 | 1674
d2nwua1 | 2 | 258 | 260
d4mypa_ | 1 | 159 | 160
d1gu7a2 | 13 | 2928 | 2941
d2jexa_ | 1 | 291 | 292
d3v8da1 | 15 | 258 | 273
d5gzca_ | 1 | 45 | 46
d5oolw_ | 11 | 33 | 44
d3pl2a_ | 7 | 687 | 694
d1i7ka_ | 10 | 859 | 869
d4z4ja1 | 2 | 64 | 66
d2y9ma1 | 2 | 270 | 272
d1gkpa1 | 4 | 89 | 93
d4po5a1 | 2 | 92 | 94
d3oida1 | 64 | 858 | 922
d2b

In [19]:
import torch

chkp = torch.load("/home/wngys/lab/DeepFold/model/model_10_400_100/model_4.pt")
print(chkp.keys())

losses = chkp["train_loss_ls"]
acces = chkp["valid_acc_ls"]
train_accs = chkp["train_acc_ls"]
b_acc = chkp["best_acc"]

print(b_acc)
print(acces)
print(train_accs)
print(losses)

dict_keys(['epoch', 'best_acc', 'model_param', 'optim_param', 'train_loss_ls', 'valid_acc_ls', 'train_acc_ls', 'valid_id_list'])
0.38
[0.34, 0.34, 0.38, 0.33, 0.36, 0.33, 0.37, 0.32, 0.36, 0.29]
[0.3, 0.2925, 0.2825, 0.3, 0.2625, 0.2975, 0.2475, 0.2875, 0.26, 0.2775]
[9121.327145934105, 8711.894099235535, 8653.299540162086, 8631.024267792702, 8603.476480603218]


In [ ]:
# 数据预处理部分
# 由于每个IDleft都要读取 对应的所有IDright对应的npy文件 大量IO速度过慢
# 将所有ID对应的数据文件保存在一个字典，一开始就加载进内存
def read_data():
    data_dir = "/home/wngys/lab/DeepFold/distance_matrix_r/distance_matrix_mine_r_3"
    data_dict = {}
    for file_name in os.listdir(data_dir):
        file_path = os.path.join(data_dir, file_name)
        data = np.load(file_path, allow_pickle=True)
        ID = file_name.split('.')[0]
        data_dict[ID] = data

    fileName = "/home/wngys/lab/DeepFold/distance_matrix_r/matrix_data.npy"
    
    np.save(fileName, data_dict)

read_data()

In [23]:
# 数据预处理部分 计算整体三个channel mean 和 std 用于normalize
import numpy as np
import torch

Dir_path = "/home/wngys/lab/DeepFold/distance_matrix_r/distance_matrix_mine_r_3"
total_mean0 = 0
total_mean1 = 0
total_mean2 = 0
total_std0 = 0
total_std1 = 0
total_std2 = 0
tfm = build_transform(in_channel=3)
num = len(os.listdir(Dir_path))
for file_name in os.listdir(Dir_path):
    file_path = os.path.join(Dir_path, file_name)
    data = np.load(file_path, allow_pickle=True)
    data = torch.from_numpy(data)
    data = tfm(data)
    total_mean0 += data[0].mean()
    total_mean1 += data[1].mean()
    total_mean2 += data[2].mean()
    total_std0 += data[0].std()
    total_std1 += data[1].std()
    total_std2 += data[2].std()
avg_mean0 = total_mean0/num
avg_mean1 = total_mean1/num
avg_mean2 = total_mean2/num

avg_std0 = total_std0/num
avg_std1 = total_std1/num
avg_std2 = total_std2/num

print(avg_mean0)
print(avg_mean1)
print(avg_mean2)
print("***********************************")
print(avg_std0)
print(avg_std1)
print(avg_std2)

tensor(0.0014, dtype=torch.float64)
tensor(-0.0248, dtype=torch.float64)
tensor(-1.9978e-06, dtype=torch.float64)
***********************************
tensor(0.9975, dtype=torch.float64)
tensor(0.9763, dtype=torch.float64)
tensor(0.9537, dtype=torch.float64)


In [22]:
file_path = "/home/wngys/lab/DeepFold/distance_matrix_r/matrix_data.npy"

dict_data = np.load(file_path, allow_pickle=True).tolist()
print(f"num:{len(dict_data.keys())}")
num = len(dict_data.keys())

total_mean0 = 0
total_mean1 = 0
total_mean2 = 0
total_std0 = 0
total_std1 = 0
total_std2 = 0

for Id in dict_data.keys():
    data = dict_data[Id]
    data = torch.from_numpy(data)
    total_mean0 += data[0].mean()
    total_mean1 += data[1].mean()
    total_mean2 += data[2].mean()
    total_std0 += data[0].std()
    total_std1 += data[1].std()
    total_std2 += data[2].std()

avg_mean0 = total_mean0/num
avg_mean1 = total_mean1/num
avg_mean2 = total_mean2/num

avg_std0 = total_std0/num
avg_std1 = total_std1/num
avg_std2 = total_std2/num

print(avg_mean0)
print(avg_mean1)
print(avg_mean2)
print("***********************************")
print(avg_std0)
print(avg_std1)
print(avg_std2)
    
    

num:14274
tensor(0.0068, dtype=torch.float64)
tensor(0.0003, dtype=torch.float64)
tensor(2.3069e-05, dtype=torch.float64)
***********************************
tensor(0.0140, dtype=torch.float64)
tensor(0.0015, dtype=torch.float64)
tensor(0.0002, dtype=torch.float64)


In [26]:
# lossFt = nn.MSELoss()

import torch
t = torch.tensor([[1,1], [2,4], [1, 2], [2, 2]], dtype = torch.float, requires_grad=True)

# lossFt = t[0][0] + 2* t[0][1]

# lossFt.backward()
# print(t.grad)

l1 = []
l2 = []
l1.append(t[1])
l1.append(t[3])
l2.append(t[0])
l2.append(t[2])

l1_c = []
l2_c = []

v = torch.tensor([1, 2], dtype=torch.float)

l1_c.append(F.cosine_similarity(v, l1[0], dim = 0))
l1_c.append(F.cosine_similarity(v, l1[1], dim = 0))
# print(l1_c)
l1_c.sort()

lossF = max(l1_c[0] - l1_c[1], torch.tensor(0,dtype=torch.float, requires_grad=True).detach())
lossF += 1
lossF.backward()
print(t.grad)
# l2_c.append(F.cosine_similarity)





RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn